Initial Data Observation

In [1]:
#QUESTIONS WE WANT TO ANSWER:

# Determine which aircraft are the lowest risk for the company to start this new business endeavor
# They are interested in purchasing and operating airplanes for commercial and private enterprises
    #1ST RECC. COMMERCIAL
    #2ND RECC. PRIVATE
    
#Make avs. Model - groupby: 'Make'
# then avg severity ratio

#remove the makes that have less than 20
    #not enough information for these to be sound
    
#average severity ratio

#Think as one of two ways
    
    
# Actionable insights that the head of the new aviation division can use to help decide which aircraft to purchase.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('../data/archive/AviationData.csv', encoding='cp1252', low_memory=False)
uscodes_df = pd.read_csv('../data/archive/USState_Codes.csv', encoding='cp1252', low_memory=False)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

In [5]:
# Replace'.' with ' '
df.columns = [c.replace('.', ' ') for c in df]

#We are only concerned with Airplanes in the Aircraft Category
df = df[df['Aircraft Category'] == 'Airplane']

#Standardizing all of the airplane makes by capitalizing.
df['Make'] = df['Make'].str.upper()
df['Air carrier'] = df['Air carrier'].str.title()

df['Weather Condition'] = df['Weather Condition'].str.upper()

#Dropping rows that are 'Amateur Built == 'Yes''
df = df[df['Amateur Built'] == 'No']

#Dropping columns that are not relevant
df = df.drop(columns=['Amateur Built', 'Injury Severity', 'Weather Condition', 'Publication Date', 'Aircraft Category', 'Latitude', 'Longitude', 
                                        'Airport Code', 'Airport Name', 'Registration Number', 'Event Id', 'Air carrier', 'Schedule', 'Broad phase of flight'])

df['FAR Description'].replace(to_replace= '121', value='Part 121', inplace=True)
df['FAR Description'].replace(to_replace= '135', value='Part 135', inplace=True)
df['FAR Description'].replace(to_replace= '137', value='Part 137', inplace=True)
df['FAR Description'].replace(to_replace= '129', value='Part 129', inplace=True)
df['FAR Description'].replace(to_replace= 'Part 137', value='Part 137: Agricultural', inplace=True)
df['FAR Description'].replace(to_replace= 'Part 135', value='Part 135: Air Taxi & Commuter', inplace=True)
df['FAR Description'].replace(to_replace= 'Part 121', value='Part 121: Air Carrier', inplace=True)
df['FAR Description'].replace(to_replace= 'Part 129', value='Part 129: Foreign', inplace=True)
df['FAR Description'].replace(to_replace= 'Part 129', value='Part 129: Foreign', inplace=True)
df['FAR Description'].replace(to_replace= 'NUSN', value='Non-U.S., Non-Commercial', inplace=True)
df['FAR Description'].replace(to_replace= 'NUSC', value='Non-U.S., Commercial', inplace=True)
df['FAR Description'].replace(to_replace= '091', value='Part 91: General Aviation', inplace=True)
df['FAR Description'].replace(to_replace= '125', value='Part 125: 20+ Pax,6000+ lbs', inplace=True)
df['FAR Description'].replace(to_replace= 'UNK', value='Unknown', inplace=True)

#Keep 'Part 91: General Aviation', 'Non-U.S., Commercial', 'Part 121: Air Carrier', 'Part 129: Foreign', 'Unknown' 
df = df[(df['FAR Description']=='Part 91: General Aviation') | (df['FAR Description']=='Non-U.S., Commercial') |
  (df['FAR Description']=='Part 121: Air Carrier') | (df['FAR Description']=='Part 129: Foreign') |
  (df['FAR Description']=='Unknown')]                 

#Dropping all rows before 1990 - Average lifespan for a plane is 30 years                
df = df[(df['Event Date'] > '1990-00-00')]

#CREATING A RATIO OF INJURIES TO UNINJURED
df_injuries = df[['Total Fatal Injuries', 'Total Serious Injuries', 'Total Minor Injuries', 'Total Uninjured']].fillna(0)
df_injuries
injuries = df_injuries['Total Fatal Injuries']*3 + df_injuries['Total Serious Injuries']*2 - df_injuries['Total Minor Injuries']
total = df_injuries['Total Fatal Injuries'] + df_injuries['Total Serious Injuries'] + df_injuries['Total Minor Injuries'] + df_injuries['Total Uninjured']
severity = injuries/total
df_injuries['Severity Ratio'] = severity*100
df_injuries.reset_index(inplace=True)
df_injuries.drop(axis=1, labels='index', inplace=True)
df_injuries

df_injuries['Make'] = df['Make'] +
df_injuries


,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Injured/Uninjured Ratio,Make
0,0.0,0.0,0.0,2.0,0.000000,NaN
1,0.0,0.0,0.0,1.0,0.000000,NaN
2,1.0,0.0,0.0,0.0,3.000000,NaN
3,0.0,0.0,0.0,2.0,0.000000,NaN
4,2.0,0.0,0.0,7.0,0.666667,NaN
...,...,...,...,...,...,...
18041,0.0,0.0,0.0,1.0,0.000000,NaN
18042,0.0,0.0,0.0,1.0,0.000000,NaN
18043,0.0,0.0,0.0,1.0,0.000000,NaN
18044,0.0,1.0,0.0,0.0,2.000000,NaN


In [6]:
df.groupby('Make')

In [7]:
#Merge double values
#aviation_df.replace(to_replace=, value= )

In [8]:
#'Engine Type' column - nothing to clean yet
#aviation_df['Engine Type'].value_counts()

In [9]:
#aviation_df['Schedule'].value_counts()
#SCHD    1497
#NSCH    1237
#UNK      256

In [10]:
#Get rid of a lot in this column
#df['Purpose of flight'].value_counts()

In [11]:
#len(aviation_df['Air carrier'].unique())

In [12]:
#THIS CATEGORY MAYBE SHOULD NOT BE CONSIDERED RELEVANT - want airplane to operate in all weather conditions
#aviation_df['Weather Condition'].value_counts()

In [13]:
#THIS CATEGORY MAYBE SHOULD NOT BE CONSIDERED RELEVANT - 
#the phase of the flgiht when the incident/accident occured should not matter
#aviation_df['Broad phase of flight'].value_counts()

In [14]:
#THIS CATEGORY MAYBE SHOULD NOT BE CONSIDERED RELEVANT - the written report of the incident
#aviation_df['Air carrier'].value_counts()

In [15]:
#aviation_df['Event Date'].value_counts()